In [1]:
# %%
# Queries database to get data with our without dates

import pandas as pd
import numpy as np
#port geopandas as gpd
from sqlalchemy import create_engine, text

import mpl_toolkits.mplot3d

#query db, return array
def query_db(include_weights=False):
    engine = create_engine("sqlite:///updated_fires_db.sqlite")
    query = ' ' 
    
    with engine.connect() as connection:
        
        #overwriting columns:
        if(include_weights):
            query = "SELECT Fires.REAL_DISCOVERY_DOY, Fires.REAL_LATITUDE, Fires.REAL_LONGITUDE, Fires.REAL_FIRE_SIZE, Fires.REAL_FIRE_YEAR, CAUSE_NATURAL, Calendar.Date_weight FROM FIRES JOIN Calendar ON Fires.DISCOVERY_DOY = Calendar.DOY AND Fires.FIRE_YEAR = Calendar.Year WHERE Fires.NWCG_GENERAL_CAUSE != 'Missing data/not specified/undetermined'"
        else:
            query = "SELECT Fires.REAL_DISCOVERY_DOY, Fires.REAL_LATITUDE, Fires.REAL_LONGITUDE, Fires.REAL_FIRE_SIZE, Fires.REAL_FIRE_YEAR, CAUSE_NATURAL FROM FIRES WHERE Fires.NWCG_GENERAL_CAUSE != 'Missing data/not specified/undetermined'" #, Calendar.Date_weight FROM FIRES JOIN Calendar ON Fires.DISCOVERY_DOY = Calendar.DOY AND Fires.FIRE_YEAR = Calendar.Year  WHERE Fires.NWCG_GENERAL_CAUSE != 'Missing data/not specified/undetermined'"

        print("query: " + query)
        result = connection.execute(text(query))
        
        data = []
        for row in result:
            a = []
            for item in row:
                a.append(item)

            data.append(a)
            # print(data)
        clean_data = np.array(data)

        # Gets raw unnormalized data for analasis
        query = "SELECT "
        regular_columns = ["DISCOVERY_DOY", "LATITUDE", "LONGITUDE", "FIRE_SIZE", "FIRE_SIZE_CLASS", "FIRE_YEAR", "STATE", "NWCG_REPORTING_AGENCY", "NWCG_GENERAL_CAUSE"]

        # adds all standard columns to output
        first_col = True
        for col in regular_columns:
            # only adds commas after the first col
            if(not first_col):
                query += ", "
            query += col
            first_col = False
            
        if(include_weights):
            query += ", Date_weight FROM FIRES JOIN Calendar ON Fires.DISCOVERY_DOY = Calendar.DOY AND Fires.FIRE_YEAR = Calendar.Year WHERE Fires.NWCG_GENERAL_CAUSE != 'Missing data/not specified/undetermined'"
            regular_columns.append("Date_weight")
        else:
            query += " FROM FIRES WHERE Fires.NWCG_GENERAL_CAUSE != 'Missing data/not specified/undetermined'"
        result = connection.execute(text(query))
        
        data = []
        for row in result:
            a = []
            for item in row:
                a.append(item)

            data.append(a)
            # print(data)
        raw_data = np.array(data)
        #print(x)
    return raw_data, clean_data, regular_columns #used_columns]

# %%


In [2]:
data, cleaned_data, column_headers = query_db(True)


query: SELECT Fires.REAL_DISCOVERY_DOY, Fires.REAL_LATITUDE, Fires.REAL_LONGITUDE, Fires.REAL_FIRE_SIZE, Fires.REAL_FIRE_YEAR, CAUSE_NATURAL, Calendar.Date_weight FROM FIRES JOIN Calendar ON Fires.DISCOVERY_DOY = Calendar.DOY AND Fires.FIRE_YEAR = Calendar.Year WHERE Fires.NWCG_GENERAL_CAUSE != 'Missing data/not specified/undetermined'


In [ ]:
# Henry Code Usage
import read

alpha = 0.11433952525429983 #read.confounding_strength_DOY(data, column_headers)
print(alpha)

#data, cleaned_data, column_headers = query_db(False)
read.do_kmean(data, cleaned_data, column_headers, "Weighted Query")
#print(read.MODIFIED_DB)